## Google Dance

In [1]:
#Importing all the libraries required for scraping

from splinter import Browser
from bs4 import BeautifulSoup 
import requests
import time
import pandas as pd
import pprint
import webbrowser
import urllib
import re
import pymongo
from geopy.geocoders import Nominatim  

In [2]:
# using the crome driver for scraping

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#browser = Browser()

In [3]:
# Url for main page
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Y5HcXZW5CdG05gKT7KzIAw&q=kids+dance+classes+new+jersey&oq=kids+dance+classes+new+jersey&gs_l=psy-ab.3...140911.143275.0.143677.6.6.0.0.0.0.271.904.0j4j1.5.0....0...1c.1.64.psy-ab..1.0.0....0.2IUp-hMGy8U#rlfi=hd:;si:;mv:[[41.0322952,-73.93508229999999],[40.534548400000006,-74.637359]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

# updating the page by clicking on search button
browser.find_by_name('btnG').first.click()

# parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

# storing all the data from the main class
Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
# search variables Name and reviews within main class using for loop
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

21
[{'name': 'Broadway Bound Dance Center Inc.'}, {'name': 'Hudson Dance Studio Edgewater'}, {'name': 'Wild Child Dance Studio'}, {'name': 'DDA Dance Academy'}, {'name': 'Blossom Sisters Dance & Performing Arts'}, {'name': 'Class Act Performing Arts Studio'}, {'name': 'Dance Palooza Studios'}, {'name': 'Starlight Dance Center'}, {'name': 'The Kennedy Dancers'}, {'name': 'Dance Power'}, {'name': 'Dance Designs Studio'}, {'name': 'Center Stage Dance Studio'}, {'name': 'Undercover Dance Factory'}, {'name': 'Broadway Bound Dance Center Inc.'}, {'name': 'Progressive Dance Studio'}, {'name': 'Next Step Broadway'}, {'name': 'The Tate Academy'}, {'name': 'Performing Arts Workshop'}, {'name': 'Professional Arts Academy'}, {'name': 'Uptown Dance Studio'}, {'name': 'Harmony Dance Center'}]


In [4]:
# Scraping Dance Classes
Data = []
# Using the geo locator for getting Lat and Long
geolocator = Nominatim(user_agent="my-application")

# getting the link for next pages 

pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

#updating each page link with specific nunmbers to go to next page

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

# parsing pages as the pages updated with above url

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    #print(len(Data))

# search variables Name, url, adress, phone no, zipcode within main class using for loop

    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Dance Classes"
        
#if the url is empty handling the exception

        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            #print(dataframe["url"])
        except:
            dataframe["url"] = '#'
            print('Cant Find the url. Entered # instead')

# scraping for phone number

        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        
# if the main directory is empty, handling the exception
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
        
# getting the address
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])

# cleaning the address based on number of delimters

            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                #    print(orgAddress)
                
# Using geolocator to find the lat and long for each address

            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)

# creating a dataframe for Dance
df1=pd.DataFrame(top_result)



1
/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=haToXc_ZB-mj_QaL15GwDA&start=20&sa=N&ved=0ahUKEwiPsIHD8Z3mAhXpUd8KHYtrBMYQ8tMDCO4C
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead
Cant Find the url. Entered # instead

In [5]:
#df1.to_excel("outputDance.xlsx")

# creating a dataframe for Dance
df1

,address,location,name,phoneNo,type,url
0,"357 Broad St %231, Bloomfield, NJ, 07003",NaN,Broadway Performing Arts,9737482787,Dance Classes,http://broadwayperformingarts.net/
1,"285 Market St, Elmwood Park, NJ, 07407","[40.9001665555556, -74.1187018888889]",Gotta Dance,,Dance Classes,http://www.gottadancenj.com/
2,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
3,"355 Eisenhower Pkwy %23100, Livingston, NJ, ...",NaN,Grooves Unlimited Dance Studio,,Dance Classes,http://www.groovesunlimiteddancestudio.com/
4,"599 Chestnut St %23 A, Union, NJ, 07083",NaN,Cherney Dance Studio,9,Dance Classes,http://cherneydancestudio.com/
5,"29 Edward Ct, Clifton, NJ, 07011","[40.88596, -74.162009]",Davis Academy of Irish Dancing,9733409026,Dance Classes,#
6,"187 Bloomfield Ave, Bloomfield, NJ, 07003","[40.7811202857143, -74.1905645714286]",Just Dancin',,Dance Classes,http://justdancinnj.com/
7,"97 Main St, Woodbridge, NJ, 07095","[40.5537899, -74.2804535]",Across the Floor,,Dance Classes,http://www.acrossthefloor.com/
8,"107 West Side Ave, Jersey City, NJ, 07305","[40.7088921020408, -74.0958224897959]",Jersey City Dance Academy,,Dance Classes,http://www.jerseycitydanceacademy.com/
9,"189 Brunswick St 2nd Floor, Jersey City, NJ,...",NaN,Jersey City Ballet,,Dance Classes,http://jerseycityballet.com/


## Google Music

In [6]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&tbm=lcl&sxsrf=ACYBGNTdFaDd4jAeC6lZKeILHtxz3pMx4A%3A1574389394556&ei=kkbXXanHIeSV_Qbz-a_wAg&q=kids+music+classes+new+jersey&oq=kids+music+classes+new+jersey&gs_l=psy-ab.12...0.0.0.163915.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.51R-_vCUfl4#rlfi=hd:;si:;mv:[[41.0328425,-73.9399237],[40.1005597,-74.3127616]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e3!2m1!1e16!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

20
[{'name': 'Musical Munchkins of New Jersey'}, {'name': 'Music Together of Bergen County'}, {'name': 'Little Rockers - Cool Kids Music Classes'}, {'name': 'The Institute of Music for Children'}, {'name': 'Joy Kids Music & Art (Joy Performing Art Center)'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Rockness Music'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'City Kids Music, LLC'}, {'name': 'Arete Music Academy'}, {'name': 'School of Rock Saddle Brook'}, {'name': 'Kidzmusik'}, {'name': 'A Sound Start'}, {'name': 'Music Together of Bergen County'}, {'name': 'Union City Music & Dance Academy'}, {'name': 'The Extension Division at Cali School of Music'}, {'name': 'Music Play Studios'}, {'name': 'Music Together of Montclair'}, {'name': 'Nutley Music Lessons'}, {'name': 'School of Rock Hoboken'}]


In [7]:
# Scraping Music Classes
Data = []
geolocator = Nominatim(user_agent="my-application")
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    #print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Music Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            #print(dataframe["url"])
        except:
            dataframe["url"] = "#"
            print('Cant Find the url. Entered # instead')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                    #print(orgAddress)
            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df2=pd.DataFrame(top_result)

1
/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=oaToXdCLLcaMgge77I_AAQ&start=20&sa=N&ved=0ahUKEwiQ4NPQ8Z3mAhVGhuAKHTv2AxgQ8tMDCMQC
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address


In [8]:
#df.to_excel("outputMusic.xlsx")

# Creating a dataframe for Music
df2

,address,location,name,phoneNo,type,url
0,"6040 Boulevard E, West New York, NJ, 07093",NaN,Music Together,2018694704,Music Classes,http://musicplaystudios.com/
1,"50 Piermont Rd, Cresskill, NJ, 07626","[40.9292453603722, -73.9634886828095]",School of Rock Cresskill,,Music Classes,http://cresskill.schoolofrock.com/
2,"411 E Clinton Ave, Tenafly, NJ, 07670","[40.9117861860619, -73.9464361870262]",JCC Thurnauer School of Music,2014081465,Music Classes,http://www.jccotp.org/thurnauer
3,"7 Vandeventer Ave, Princeton, NJ, 08542","[40.3509115, -74.6573015]",Music Together Princeton Lab School,8,Music Classes,http://www.musictogetherprinceton.com/
4,"622 NJ-10 unit 25, Whippany, NJ, 07981",NaN,Sharon's Studio of Dance & Music,1030,Music Classes,http://www.sharonsdance.com/
5,"145 Vreeland Ave, Nutley, NJ, 07110","[40.8233143333333, -74.1569040666667]",Tempo! Music Therapy Services,9736612060,Music Classes,http://www.tempotherapy.com/
6,"125 Valley Rd, Montclair, NJ, 07042","[40.821821291455, -74.2199718839158]",School of Rock Montclair,,Music Classes,http://www.montclair.schoolofrock.com/
7,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]",Newark School of the Arts,,Music Classes,http://www.newarkschoolofthearts.org/
8,"1055 Hamburg Turnpike Unit A-1, Wayne, NJ, 0...",NaN,School of Rock Wayne,,Music Classes,https://wayne.schoolofrock.com/
9,"1259 US-46, Parsippany, NJ, 07054","[40.8685123, -74.4145893]",ShreeRaga Carnatic Vocal Music School,5,Music Classes,http://www.shreeraga.com/


## Google Sports

In [9]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=05bdXaEyoqX9BoGbkuAK&q=kids+sports+classes+new+jersey&oq=kids+sports+classes+new+jersey&gs_l=psy-ab.12...13265.16654.0.22362.7.7.0.0.0.0.83.504.7.7.0....0...1c.1.64.psy-ab..0.0.0....0.EEUl6U13Yy0#rlfi=hd:;si:;mv:[[41.1122716,-73.93472659999999],[40.3136717,-74.6914486]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:3'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'The Little Gym of Hasbrouck Heights'}, {'name': 'Camp Pinebrook'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'The Little Gym of Hasbrouck Heights'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'The Little Gym of Montclair'}, {'name': 'The GYM For Kids'}, {'name': 'New York Sports Clubs'}, {'name': 'Karate For Youth Child Care'}]


In [10]:
# Scraping Sports Classes
Data = []
geolocator = Nominatim(user_agent="my-application")
pageData = soup.find_all(class_='std')
#print( soup.find("div", class_="std")["aria-label"] )

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    #print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    #print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Sports Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            #print(dataframe["url"])
        except:
            dataframe["url"] = "#"
            print('Cant Find the url. Entered # instead')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                    #print(orgAddress)
                    
            try:
                locForLatLong = dataframe["address"]# + dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
        top_result.append(dataframe)

#print(top_result)
df3=pd.DataFrame(top_result)

1
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url. Entered # instead
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address


In [11]:
#df.to_excel("outputSports.xlsx")

# creating a dataframe for sports
df3

,address,location,name,phoneNo,type,url
0,"123 Main St, Hackensack, NJ, 07601","[40.882028184209, -74.0437089023774]",Poise Martial Arts of Hackensack,10,Sports Classes,http://www.martialartshackensack.com/
1,"1121 NJ-34 Suite O, Matawan, NJ, 07747",NaN,The Little Gym of Aberdeen,7327720060,Sports Classes,http://www.thelittlegym.com/aberdeennj
2,"198 Bellevue Ave, Montclair, NJ, 07043","[40.840584, -74.20581]",Inclusive Movement Center,330,Sports Classes,http://inclusivemovementcenter.com/
3,"1330 Springfield Ave, New Providence, NJ, 07974","[40.705364732823, -74.4076033327866]",My Gym New Providence,9086652122,Sports Classes,http://www.mygym.com/newprovidence
4,"2040 Springdale Rd, Cherry Hill, NJ, 08003","[39.9120628, -74.9656516]",Kid Fit NJ,9,Sports Classes,https://www.nj-kidfit.com/
5,"120 E Kings Hwy %23403, Maple Shade Township...",NaN,LittleSport,,Sports Classes,https://www.littlesport.net/
6,"545 Morris Ave, Summit, NJ, 07901","[40.728246, -74.375629]",The Little Gym of Summit,9086089400,Sports Classes,http://www.thelittlegym.com/summitnj
7,"260 E State RT 4, Paramus, NJ, 07652",NaN,SafeSplash Swim School - Paramus,4,Sports Classes,https://www.safesplash.com/locations/paramus
8,"40 Eisenhower Dr %23102, Paramus, NJ, 07652",NaN,DDA Dance Academy,2018197989,Sports Classes,#
9,"42 City Pl, Edgewater, NJ, 07020","[40.8075702819484, -73.9897644932853]",Sweat NJ KIDS,1030,Sports Classes,#


## Merged all three dataframe into one DF

In [12]:
# merging all 3 dataframe for music, Dannce and Sports

df = df1.append(df2, ignore_index=True)
df = df.append(df3, ignore_index=True)

# Cleaning the dataframet to drop null values for location

df = df.dropna(axis=0, subset=['location'])
df = df[df.location != 0]
df

,address,location,name,phoneNo,type,url
1,"285 Market St, Elmwood Park, NJ, 07407","[40.9001665555556, -74.1187018888889]",Gotta Dance,,Dance Classes,http://www.gottadancenj.com/
2,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
5,"29 Edward Ct, Clifton, NJ, 07011","[40.88596, -74.162009]",Davis Academy of Irish Dancing,9733409026,Dance Classes,#
6,"187 Bloomfield Ave, Bloomfield, NJ, 07003","[40.7811202857143, -74.1905645714286]",Just Dancin',,Dance Classes,http://justdancinnj.com/
7,"97 Main St, Woodbridge, NJ, 07095","[40.5537899, -74.2804535]",Across the Floor,,Dance Classes,http://www.acrossthefloor.com/
8,"107 West Side Ave, Jersey City, NJ, 07305","[40.7088921020408, -74.0958224897959]",Jersey City Dance Academy,,Dance Classes,http://www.jerseycitydanceacademy.com/
10,"10 Durand Rd, Maplewood, NJ, 07040","[40.7319036798282, -74.2762489369301]",Lydia Johnson Dance School,4,Dance Classes,http://www.lydiajohnsondanceschool.com/
11,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]",Newark School of the Arts,,Dance Classes,http://www.newarkschoolofthearts.org/
12,"15 Seminary Ave, Chester, NJ, 07930","[40.7836976632736, -74.6972963591172]",Art of Dance,,Dance Classes,http://www.artofdance.org/
13,"2 Villa Dr, Edison, NJ, 08820","[40.599712, -74.354303]",Edison School of Music and Dance,,Dance Classes,http://www.edisonmusicanddance.com/


In [13]:
# exporting to excel
df.to_excel("outputKidsActivities.xlsx") 

In [14]:
# reseting the index
df=df.reset_index(drop=True)
df

,address,location,name,phoneNo,type,url
0,"285 Market St, Elmwood Park, NJ, 07407","[40.9001665555556, -74.1187018888889]",Gotta Dance,,Dance Classes,http://www.gottadancenj.com/
1,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
2,"29 Edward Ct, Clifton, NJ, 07011","[40.88596, -74.162009]",Davis Academy of Irish Dancing,9733409026,Dance Classes,#
3,"187 Bloomfield Ave, Bloomfield, NJ, 07003","[40.7811202857143, -74.1905645714286]",Just Dancin',,Dance Classes,http://justdancinnj.com/
4,"97 Main St, Woodbridge, NJ, 07095","[40.5537899, -74.2804535]",Across the Floor,,Dance Classes,http://www.acrossthefloor.com/
5,"107 West Side Ave, Jersey City, NJ, 07305","[40.7088921020408, -74.0958224897959]",Jersey City Dance Academy,,Dance Classes,http://www.jerseycitydanceacademy.com/
6,"10 Durand Rd, Maplewood, NJ, 07040","[40.7319036798282, -74.2762489369301]",Lydia Johnson Dance School,4,Dance Classes,http://www.lydiajohnsondanceschool.com/
7,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]",Newark School of the Arts,,Dance Classes,http://www.newarkschoolofthearts.org/
8,"15 Seminary Ave, Chester, NJ, 07930","[40.7836976632736, -74.6972963591172]",Art of Dance,,Dance Classes,http://www.artofdance.org/
9,"2 Villa Dr, Edison, NJ, 08820","[40.599712, -74.354303]",Edison School of Music and Dance,,Dance Classes,http://www.edisonmusicanddance.com/


In [15]:
df = df.reset_index()
df

,index,address,location,name,phoneNo,type,url
0,0,"285 Market St, Elmwood Park, NJ, 07407","[40.9001665555556, -74.1187018888889]",Gotta Dance,,Dance Classes,http://www.gottadancenj.com/
1,1,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
2,2,"29 Edward Ct, Clifton, NJ, 07011","[40.88596, -74.162009]",Davis Academy of Irish Dancing,9733409026,Dance Classes,#
3,3,"187 Bloomfield Ave, Bloomfield, NJ, 07003","[40.7811202857143, -74.1905645714286]",Just Dancin',,Dance Classes,http://justdancinnj.com/
4,4,"97 Main St, Woodbridge, NJ, 07095","[40.5537899, -74.2804535]",Across the Floor,,Dance Classes,http://www.acrossthefloor.com/
5,5,"107 West Side Ave, Jersey City, NJ, 07305","[40.7088921020408, -74.0958224897959]",Jersey City Dance Academy,,Dance Classes,http://www.jerseycitydanceacademy.com/
6,6,"10 Durand Rd, Maplewood, NJ, 07040","[40.7319036798282, -74.2762489369301]",Lydia Johnson Dance School,4,Dance Classes,http://www.lydiajohnsondanceschool.com/
7,7,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]",Newark School of the Arts,,Dance Classes,http://www.newarkschoolofthearts.org/
8,8,"15 Seminary Ave, Chester, NJ, 07930","[40.7836976632736, -74.6972963591172]",Art of Dance,,Dance Classes,http://www.artofdance.org/
9,9,"2 Villa Dr, Edison, NJ, 08820","[40.599712, -74.354303]",Edison School of Music and Dance,,Dance Classes,http://www.edisonmusicanddance.com/


In [16]:
# Rename Index columns to "_id"
df = df.rename(columns={'index':'_id'})
df.head()

,_id,address,location,name,phoneNo,type,url
0,0,"285 Market St, Elmwood Park, NJ, 07407","[40.9001665555556, -74.1187018888889]",Gotta Dance,,Dance Classes,http://www.gottadancenj.com/
1,1,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
2,2,"29 Edward Ct, Clifton, NJ, 07011","[40.88596, -74.162009]",Davis Academy of Irish Dancing,9733409026,Dance Classes,#
3,3,"187 Bloomfield Ave, Bloomfield, NJ, 07003","[40.7811202857143, -74.1905645714286]",Just Dancin',,Dance Classes,http://justdancinnj.com/
4,4,"97 Main St, Woodbridge, NJ, 07095","[40.5537899, -74.2804535]",Across the Floor,,Dance Classes,http://www.acrossthefloor.com/


In [17]:
# Add 1 to each "_id" because first activity must start with 1 not 0.  Starting with "_id" = 0 will throw an error
df['_id'] = df['_id'] + 1 
df

,_id,address,location,name,phoneNo,type,url
0,1,"285 Market St, Elmwood Park, NJ, 07407","[40.9001665555556, -74.1187018888889]",Gotta Dance,,Dance Classes,http://www.gottadancenj.com/
1,2,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
2,3,"29 Edward Ct, Clifton, NJ, 07011","[40.88596, -74.162009]",Davis Academy of Irish Dancing,9733409026,Dance Classes,#
3,4,"187 Bloomfield Ave, Bloomfield, NJ, 07003","[40.7811202857143, -74.1905645714286]",Just Dancin',,Dance Classes,http://justdancinnj.com/
4,5,"97 Main St, Woodbridge, NJ, 07095","[40.5537899, -74.2804535]",Across the Floor,,Dance Classes,http://www.acrossthefloor.com/
5,6,"107 West Side Ave, Jersey City, NJ, 07305","[40.7088921020408, -74.0958224897959]",Jersey City Dance Academy,,Dance Classes,http://www.jerseycitydanceacademy.com/
6,7,"10 Durand Rd, Maplewood, NJ, 07040","[40.7319036798282, -74.2762489369301]",Lydia Johnson Dance School,4,Dance Classes,http://www.lydiajohnsondanceschool.com/
7,8,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]",Newark School of the Arts,,Dance Classes,http://www.newarkschoolofthearts.org/
8,9,"15 Seminary Ave, Chester, NJ, 07930","[40.7836976632736, -74.6972963591172]",Art of Dance,,Dance Classes,http://www.artofdance.org/
9,10,"2 Villa Dr, Edison, NJ, 08820","[40.599712, -74.354303]",Edison School of Music and Dance,,Dance Classes,http://www.edisonmusicanddance.com/


In [18]:
# adding to directory
split_data = df.to_dict('records')
split_data

[{'_id': 1,
  'address': ' 285 Market St,  Elmwood Park, NJ, 07407',
  'location': [40.9001665555556, -74.1187018888889],
  'name': 'Gotta Dance',
  'phoneNo': '',
  'type': 'Dance Classes',
  'url': 'http://www.gottadancenj.com/'},
 {'_id': 2,
  'address': ' 234 Franklin Ave,  Nutley, NJ, 07110',
  'location': [40.815737, -74.161944],
  'name': 'Ellis Dance Studio',
  'phoneNo': '',
  'type': 'Dance Classes',
  'url': 'http://ellisdancestudios.com/'},
 {'_id': 3,
  'address': ' 29 Edward Ct,  Clifton, NJ, 07011',
  'location': [40.88596, -74.162009],
  'name': 'Davis Academy of Irish Dancing',
  'phoneNo': '9733409026',
  'type': 'Dance Classes',
  'url': '#'},
 {'_id': 4,
  'address': ' 187 Bloomfield Ave,  Bloomfield, NJ, 07003',
  'location': [40.7811202857143, -74.1905645714286],
  'name': "Just Dancin'",
  'phoneNo': '',
  'type': 'Dance Classes',
  'url': 'http://justdancinnj.com/'},
 {'_id': 5,
  'address': ' 97 Main St,  Woodbridge, NJ, 07095',
  'location': [40.5537899, -74.2

In [19]:
# Ensure that all types are objects
df.dtypes

_id          int64
address     object
location    object
name        object
phoneNo     object
type        object
url         object
dtype: object

In [20]:
# Set up the MongoDB connection through pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# Create DB

db = myclient["KidsRidedb3"]

# Create collection and insert all the data into the MongoDB

mycol = db["activityData30"]
mycol.drop()
mycol.insert_many(split_data)

Combined Data now stored in MongoDB database. 
Check it with MongoDB Compass or use mongod to see the Kidsridedb3 database and the activityData collections
###### Run Python Flask Application now